## Generate MetaCyc-dependendt files

### protein_complexes.txt

In [1]:
import cobra
import os
import pandas as pd
import numpy as np
import urllib
import pythoncyc as pc
import re
from Bio.SeqUtils import seq3
from Bio import Seq

## Metabolites

met_output_file = 'metabolites.txt'
m_model_file = './iYO844.json'

m_model = cobra.io.load_json_model(m_model_file)

## Fix of gene reaction rules
m_model.reactions.ACCOAC.gene_reaction_rule = '(BSU29200 and BSU29210) or BSU24350 or BSU24340 or BSU22440'
m_model.reactions.PDH.gene_reaction_rule = '(BSU14580 and BSU14590) or BSU14600 or BSU14610'
m_model.reactions.ACTD2.gene_reaction_rule = 'BSU08060 and BSU08070'
m_model.reactions.get_by_id('AIRC1').gene_reaction_rule = 'BSU06420 or BSU06430'
m_model.reactions.get_by_id('ANS').gene_reaction_rule = 'BSU00750 or BSU22680'
m_model.reactions.get_by_id('PRFGS_1').gene_reaction_rule = 'BSU06480 or BSU06470'
m_model.reactions.get_by_id('RNDR1').gene_reaction_rule = 'BSU17380 and BSU17390'
m_model.reactions.get_by_id('RNDR2').gene_reaction_rule = 'BSU17380 and BSU17390'
m_model.reactions.get_by_id('RNDR3').gene_reaction_rule = 'BSU17380 and BSU17390'
m_model.reactions.get_by_id('RNDR4').gene_reaction_rule = 'BSU17380 and BSU17390'


In [2]:
def frameid_to_str(frameid):
    string = str(frameid)
    string = string[1:len(string)-1]
    return string

In [3]:
bsub = pc.select_organism('bsub')
ecoli = pc.select_organism('ecoli')

In [4]:
PROTEINS = bsub.proteins

In [5]:
filename = 'protein_complexes.txt'
file = open(filename,'w')
CPLX_list = []
genes_all_cplxs = []

protein_complexes_dict = dict()

for cplx in PROTEINS.instances:
    
    cplx_string = frameid_to_str(cplx.frameid)
    string = cplx_string + '\t' + 'default_name' + '\t'

    
    genes_of_cplx = pc.PGDB.genes_of_protein(bsub,cplx)
    genes_of_cplx_comp = []
    grRule_string = ''
    if genes_of_cplx:
        
        for gene in genes_of_cplx:
            gene_string = frameid_to_str(gene)
            if cplx_string not in protein_complexes_dict.keys():
                protein_complexes_dict[cplx_string] = []
            protein_complexes_dict[cplx_string].append(gene_string)

            genes_of_cplx_comp.append(gene_string)
            string = string + gene_string + '(' + stoichiometry + ')' + ' AND '
            grRule_string = grRule_string + gene_string + '()' + ' AND '


        string = string[0:len(string)-5]
        grRule_string = grRule_string[0:len(grRule_string)-5]
        
        string = string + '\t' + 'M_protein_recon' + '\n'

        
        file.write(string)

        CPLX_list.append(cplx_string)
        genes_all_cplxs.append(genes_of_cplx_comp)
    
file.close()
protein_complexes_dict

{'CPLX8J2-24': ['BSU06020', 'BSU06030'],
 'BSU08300-MONOMER': ['BSU08300'],
 'BSU09120-MONOMER': ['BSU09120'],
 'CPLX8J2-27': ['BSU22740', 'BSU22760'],
 'CPLX8J2-20': ['BSU33430', 'BSU33440'],
 'CPLX8J2-21': ['BSU28250', 'BSU28260'],
 'CPLX8J2-22': ['BSU14890', 'BSU14900', 'BSU14910', 'BSU14920'],
 'CPLX8J2-25': ['BSU36850', 'BSU36860', 'BSU36870'],
 'BSU26410-MONOMER': ['BSU26410'],
 'BSU21329-MONOMER': ['BSU21329'],
 'CPLX8J2-28': ['BSU36640', 'BSU36650', 'BSU36660'],
 'BSU28720-MONOMER': ['BSU28720'],
 'BSU40800-MONOMER': ['BSU40800'],
 'BSU40870-MONOMER': ['BSU40870'],
 'BSU38250-MONOMER': ['BSU38250'],
 'BSU17780-MONOMER': ['BSU17780'],
 'BSU39970-MONOMER': ['BSU39970'],
 'BSU25960-MONOMER': ['BSU25960'],
 'BSU30170-MONOMER': ['BSU30170'],
 'BSU36360-MONOMER': ['BSU36360'],
 'BSU06440-MONOMER': ['BSU06440'],
 'BSU38430-MONOMER': ['BSU38430'],
 'BSU30660-MONOMER': ['BSU30660'],
 'BSU34370-MONOMER': ['BSU34370'],
 'BSU00920-MONOMER': ['BSU00920'],
 'CPLX8J2-23': ['BSU28300', 'BSU283

### enzyme_reaction_association.txt

In [6]:
filename = 'enzyme_reaction_association.txt'
prot_cplx_filename = 'protein_complexes.txt'

file = open(filename,'w')
prot_cplx_file = open(prot_cplx_filename,'a')

standard_gene_length = 8

enz_rxn_assoc_list = []
artificial_cplxs = []
artificial_id = 0;
for reaction in m_model.reactions:
    if not (reaction.id[0:3] == 'EX_') and not (reaction.id[0:3] == 'DM_'):
        # Skip empty rules, these reactions are assigned to CPLX_dummy
        if not reaction.gene_reaction_rule:
            print reaction.id, ' lacks gene rules'
            continue
            
        string = str(reaction.id) + '\t' 

        rule_string = str(reaction.gene_reaction_rule)
            
        if rule_string:
            rule_string = rule_string.replace('(','')
            rule_string = rule_string.replace(')','')
            rule_list = rule_string.split(' or ')
            enz_rxn_assoc = []

            reaction_cplx_list = []
            for rule in rule_list:
                rule_gene_list = rule.split(' and ')

                for index in range(0,len(genes_all_cplxs)-1):
                    ref_rule = genes_all_cplxs[index]

                    if set(ref_rule) == set(rule_gene_list):
                        rule_cplx = CPLX_list[index]
                        reaction_cplx_list.append(rule_cplx)

            enz_rxn_assoc_list.append(reaction_cplx_list)
            string = str(reaction.id) + '\t'

            if reaction_cplx_list:
                for cplx in reaction_cplx_list:
                    string = string + cplx + ' OR '
            else:
                for rule in rule_list:
                    stoichiometry_string = ''
                    if len(rule) == standard_gene_length:
                        artificial_cplx = rule + '-MONOMER'
                        stoichiometry_string = rule + '(1)'
                    else:
                        artificial_id = artificial_id + 1
                        artificial_cplx = 'CPLX000-' + str(artificial_id)
                        cplx_gene_list = rule.split(' and ')

                        for gene in cplx_gene_list:
                            stoichiometry_string = stoichiometry_string + gene + '(1)' + ' AND '
                        stoichiometry_string = stoichiometry_string[0:len(stoichiometry_string)-4]
                    string = string + artificial_cplx + ' OR '
                    
                    if artificial_cplx not in artificial_cplxs:
                        artificial_cplxs.append(artificial_cplx)
                        prot_cplx_file.write(artificial_cplx + '\t' + 'default_name' + '\t' + stoichiometry_string + '\t' + 'M_protein_recon' + '\n')
            
        string = string[0:len(string)-4]
        string = string + '\n'
        file.write(string)
    
file.close() 
prot_cplx_file.close()
artificial_cplxs = list(set(artificial_cplxs))



2HXMPt6  lacks gene rules
2PGLYCt6  lacks gene rules
2PGt6  lacks gene rules
3AMPt6  lacks gene rules
3CMPt6  lacks gene rules
3GMPt6  lacks gene rules
3PGt6  lacks gene rules
3UMPt6  lacks gene rules
5MTRt2  lacks gene rules
6PGCt6  lacks gene rules
ABTt_1  lacks gene rules
ACACt2  lacks gene rules
ACMANAt2  lacks gene rules
ACNAMt2  lacks gene rules
ACNPLYS  lacks gene rules
ACt2r  lacks gene rules
ADEt2  lacks gene rules
ALLTNt2r  lacks gene rules
ALLTAHr  lacks gene rules
AMPt6  lacks gene rules
ARABDI  lacks gene rules
ARAB_Dt  lacks gene rules
ARGKr  lacks gene rules
ARGPt6  lacks gene rules
ASNt2r  lacks gene rules
ASPt2r  lacks gene rules
ATPM  lacks gene rules
BIOMASS_BS_10  lacks gene rules
BTDt6_RR  lacks gene rules
CHORt2  lacks gene rules
CITRt2r  lacks gene rules
CMPt6  lacks gene rules
CO2t  lacks gene rules
CYSGLYt2ir  lacks gene rules
CYSTabc  lacks gene rules
CYSabc  lacks gene rules
DEXTRINt2  lacks gene rules
DHAt  lacks gene rules
DJENKabc  lacks gene rules
DTMPt6 

## TUs_from_bsubcyc.txt

In [7]:
from Bio import SeqIO

filename = 'TUs_from_bsubcyc.txt'
file = open(filename,'w')

gb_filename = 'NC_000964.gb'   
gb_file = SeqIO.read(gb_filename, 'gb')
full_seq = str(gb_file.seq)
element_types={'CDS', 'rRNA','tRNA', 'ncRNA'}

head_string = 'TU_id' + '\t' + 'start' + '\t' + 'stop' + '\t' + 'tss' + '\t' + 'strand' + '\t' + 'rho_dependent' + '\t' + 'sigma' + '\n'
file.write(head_string)
TUs = bsub.transcription_units
for tu_PFrame in TUs.instances:
    tu_data = pc.PToolsFrame.PFrame.get_frame_data(tu_PFrame)
    
    ## Start and stop
    positions = []
    for gene_fid in tu_data.components:
        if 'BSU' in gene_fid:
            gene_PFrame = pc.PToolsFrame.PFrame(gene_fid,bsub,getFrameData=False, isClass=False)
            gene_data = pc.PToolsFrame.PFrame.get_frame_data(gene_PFrame)
            positions.append(gene_data.left_end_position)
            positions.append(gene_data.right_end_position)
            gene_id = gene_fid.replace('BSU','BSU_')
    if positions:
        tu_start = min(positions)
        tu_stop = max(positions)
    else:
        tu_start = 0
        tu_stop = 0
    
    tu_tss = tu_stop
    tu_id = tu_data.frameid + '_from_RpoD_mono'
    tu_id = tu_id.replace('|','')
    tu_id = tu_id.replace('-','_')
    
    ## Strand    
    gene_id = gene_id[1:len(gene_id)-1] ## Only use one gene. The others should have the same direction.
    for feature in gb_file.features:
        if feature.type not in element_types or 'pseudo' in feature.qualifiers:
            continue
        if feature.qualifiers["locus_tag"][0] == gene_id:
            strand = '+' if feature.location.strand == 1 else '-'
    
    ##
    string = str(tu_id) + '\t' + str(tu_start) + '\t' + str(tu_stop) + '\t' + str(tu_tss) + '\t' + str(strand) + '\t' + 'True' + '\t' + 'RpoD_mono' + '\n'
    
    file.write(string)
    
file.close()

In [8]:
for instance in bsub.transcription_units.instances:
    if (instance.frameid == '|TU8J2-1577|') or (instance.frameid == '|TU8J2-467|'):
        tu_data = pc.PToolsFrame.PFrame.get_frame_data(tu_PFrame)
        print instance.frameid, tu_data.components

|TU8J2-1577| [u'|BS8J2-422|', u'|PM8J2-611|', u'|TERM8J2-736|', u'|BSU32470|', u'|BSU32480|', u'|BSU32490|', u'|BSU32500|', u'|BSU32510|']
|TU8J2-467| [u'|BS8J2-422|', u'|PM8J2-611|', u'|TERM8J2-736|', u'|BSU32470|', u'|BSU32480|', u'|BSU32490|', u'|BSU32500|', u'|BSU32510|']


## trna_to_codon dictionary

In [9]:
DNA_to_codon_table = {'TTT': 'F',
 'TTC': 'F',
 'TTA': 'L',
 'TTG': 'L',
 'TCT': 'S',
 'TCC': 'S',
 'TCA': 'S',
 'TCG': 'S',
 'TAT': 'Y',
 'TAC': 'Y',
 'TAA': '*',
 'TAG': '*',
 'TGT': 'C',
 'TGC': 'C',
 'TGA': '*',
 'TGG': 'W',
 'CTT': 'L',
 'CTC': 'L',
 'CTA': 'L',
 'CTG': 'L',
 'CCT': 'P',
 'CCC': 'P',
 'CCA': 'P',
 'CCG': 'P',
 'CAT': 'H',
 'CAC': 'H',
 'CAA': 'Q',
 'CAG': 'Q',
 'CGT': 'R',
 'CGC': 'R',
 'CGA': 'R',
 'CGG': 'R',
 'ATT': 'I',
 'ATC': 'I',
 'ATA': 'I',
 'ATG': 'M',
 'ACT': 'T',
 'ACC': 'T',
 'ACA': 'T',
 'ACG': 'T',
 'AAT': 'N',
 'AAC': 'N',
 'AAA': 'K',
 'AAG': 'K',
 'AGT': 'S',
 'AGC': 'S',
 'AGA': 'R',
 'AGG': 'R',
 'GTT': 'V',
 'GTC': 'V',
 'GTA': 'V',
 'GTG': 'V',
 'GCT': 'A',
 'GCC': 'A',
 'GCA': 'A',
 'GCG': 'A',
 'GAT': 'D',
 'GAC': 'D',
 'GAA': 'E',
 'GAG': 'E',
 'GGT': 'G',
 'GGC': 'G',
 'GGA': 'G',
 'GGG': 'G'}

tRNA_to_codon_table = dict()
for key in DNA_to_codon_table:
    aa_id = DNA_to_codon_table[key]
    aa_id = seq3(aa_id)
    key_tRNA = Seq.transcribe(key)
    tRNA_to_codon_table[key_tRNA] = aa_id
    

tRNA_to_codon_table

{'AAA': 'Lys',
 'AAC': 'Asn',
 'AAG': 'Lys',
 'AAU': 'Asn',
 'ACA': 'Thr',
 'ACC': 'Thr',
 'ACG': 'Thr',
 'ACU': 'Thr',
 'AGA': 'Arg',
 'AGC': 'Ser',
 'AGG': 'Arg',
 'AGU': 'Ser',
 'AUA': 'Ile',
 'AUC': 'Ile',
 'AUG': 'Met',
 'AUU': 'Ile',
 'CAA': 'Gln',
 'CAC': 'His',
 'CAG': 'Gln',
 'CAU': 'His',
 'CCA': 'Pro',
 'CCC': 'Pro',
 'CCG': 'Pro',
 'CCU': 'Pro',
 'CGA': 'Arg',
 'CGC': 'Arg',
 'CGG': 'Arg',
 'CGU': 'Arg',
 'CUA': 'Leu',
 'CUC': 'Leu',
 'CUG': 'Leu',
 'CUU': 'Leu',
 'GAA': 'Glu',
 'GAC': 'Asp',
 'GAG': 'Glu',
 'GAU': 'Asp',
 'GCA': 'Ala',
 'GCC': 'Ala',
 'GCG': 'Ala',
 'GCU': 'Ala',
 'GGA': 'Gly',
 'GGC': 'Gly',
 'GGG': 'Gly',
 'GGU': 'Gly',
 'GUA': 'Val',
 'GUC': 'Val',
 'GUG': 'Val',
 'GUU': 'Val',
 'UAA': 'Ter',
 'UAC': 'Tyr',
 'UAG': 'Ter',
 'UAU': 'Tyr',
 'UCA': 'Ser',
 'UCC': 'Ser',
 'UCG': 'Ser',
 'UCU': 'Ser',
 'UGA': 'Ter',
 'UGC': 'Cys',
 'UGG': 'Trp',
 'UGU': 'Cys',
 'UUA': 'Leu',
 'UUC': 'Phe',
 'UUG': 'Leu',
 'UUU': 'Phe'}

In [10]:
def get_key(my_dict, val): 
    key_list = []
    for key, value in my_dict.items(): 
         if val == value: 
                key_list.append(key)
    return key_list



In [11]:
trna_to_codon = dict()
for tRNA_PFrame in bsub.tRNAs.instances:
    tRNA_data = pc.PToolsFrame.PFrame.get_frame_data(tRNA_PFrame)
    
    tRNA_id = str(tRNA_data.frameid)
    tRNA_id = tRNA_id.replace('|','')
    tRNA_id = tRNA_id.replace('-tRNA','')
    tRNA_id = tRNA_id.replace('TRNA','tRNA')
    
    aa_id = tRNA_data.common_name
    aa_id = aa_id[len(aa_id)-3:len(aa_id)]
    
    codon_list = get_key(tRNA_to_codon_table, aa_id)
    
    trna_to_codon[tRNA_id] = codon_list
    
trna_to_codon
    

{'BSU_tRNA_1': ['UUU', 'UUC'],
 'BSU_tRNA_10': ['AUG'],
 'BSU_tRNA_11': ['GAA', 'GAG'],
 'BSU_tRNA_12': ['GUU', 'GUC', 'GUG', 'GUA'],
 'BSU_tRNA_13': ['ACC', 'ACA', 'ACG', 'ACU'],
 'BSU_tRNA_14': ['AAG', 'AAA'],
 'BSU_tRNA_15': ['CUU', 'CUG', 'CUA', 'CUC', 'UUG', 'UUA'],
 'BSU_tRNA_16': ['GGU', 'GGG', 'GGA', 'GGC'],
 'BSU_tRNA_17': ['CUU', 'CUG', 'CUA', 'CUC', 'UUG', 'UUA'],
 'BSU_tRNA_18': ['AGG', 'AGA', 'CGA', 'CGG', 'CGC', 'CGU'],
 'BSU_tRNA_19': ['CCG', 'CCA', 'CCU', 'CCC'],
 'BSU_tRNA_2': ['GAU', 'GAC'],
 'BSU_tRNA_20': ['GCA', 'GCG', 'GCC', 'GCU'],
 'BSU_tRNA_21': ['AUG'],
 'BSU_tRNA_22': ['GAU', 'GAC'],
 'BSU_tRNA_23': ['AAC', 'AAU'],
 'BSU_tRNA_24': ['ACC', 'ACA', 'ACG', 'ACU'],
 'BSU_tRNA_25': ['GGU', 'GGG', 'GGA', 'GGC'],
 'BSU_tRNA_26': ['AGG', 'AGA', 'CGA', 'CGG', 'CGC', 'CGU'],
 'BSU_tRNA_27': ['CCG', 'CCA', 'CCU', 'CCC'],
 'BSU_tRNA_28': ['GCA', 'GCG', 'GCC', 'GCU'],
 'BSU_tRNA_29': ['AAC', 'AAU'],
 'BSU_tRNA_3': ['GAA', 'GAG'],
 'BSU_tRNA_30': ['AGC', 'AGU', 'UCU', 'UCG'

In [12]:
tRNA_data

common_name,tRNA-Leu
creation_date,3465237858
creator,|keseler|
frameid,|BSU_TRNA_57-tRNA|
gene,[u'|BSU_TRNA_57|']
modified_form,[u'|charged-BSU_TRNA_57-tRNA|']
names,[u'an uncharged tRNA']
overview_node_shape,|TEE|
pgdb,
schema_p,True
synonym_slots,"[u'|ABBREV-NAME|', u'|SYNONYMS|', u'|ACCESSION-1|']"


## protein_modification.txt

In [13]:
filename = 'protein_modification.txt'
file = open(filename,'w')
file.write('Modified_enzyme' + '\t' + 'Core_enzyme' + '\t' + 'Modifications' + '\t' + 'Source' + '\n')
RXNS = bsub.enzymatic_reactions

add_cofactors_list = []
cofactors = {'|ZN+2|':'zn2','|CU+2|':'cu2','|NAD|':'nad','|FAD|':'fad','|MG+2|':'mg2','|CA+2|':'ca2',
             '|Reduced-2Fe-2S-Ferredoxins|':'2fe2s','|Oxidized-ferredoxins|':'2fe2s',
             '|Reduced-ferredoxins|':'2fe2s','|NADP|':'nad','|PYRIDOXAL_PHOSPHATE|':'pydx5p',
             '|ADENOSYLCOBALAMIN|':'adocbl','|BIOTIN|':'btn','|CPD-7|':'4fe4s','|COB-I-ALAMIN|':'cbl1',
             '|Hemes|':'heme','|MN+2|':'mn2','|HCO3|':'hco3','|CPD-6|':'2fe2s','|FMN|':'fmn',
             '|TETRA-H-BIOPTERIN|':'tbpt'}

cplx_cofactor_dict = dict()
for rxn in RXNS.instances:
    rxn_data = pc.PToolsFrame.PFrame.get_frame_data(rxn)
    cplx = rxn_data.enzyme
    cofactors_of_rxn = pc.PGDB.cofactors_and_pgroups_of_enzrxn(bsub,rxn)
    if cofactors_of_rxn:
        cplx = str(cplx[1:len(cplx)-1])
        #print rxn.frameid, cplx, cofactors_of_rxn
        if (cplx not in cplx_cofactor_dict.keys()):
            cplx_cofactor_dict[cplx] = []
        for cofactor in cofactors_of_rxn:
            try:
                if (cofactors[cofactor] not in cplx_cofactor_dict[cplx]):
                    cplx_cofactor_dict[cplx].append(cofactors[cofactor])
            except:
                add_cofactors_list.append(cofactor)

for cplx in cplx_cofactor_dict.keys():
    cofactor_string = ''
    mod_string = ''
    for cofactor in cplx_cofactor_dict[cplx]:
        cofactor_string = cofactor_string + cofactor + '()' + ' AND '
        mod_string = mod_string + '_mod_' + cofactor
    cofactor_string = cofactor_string[0:len(cofactor_string)-5]
    
    string = cplx + mod_string +'\t' + cplx + '\t' + cofactor_string + '\t' + 'M_protein_recon' + '\n'
    
    file.write(string)
file.close()

print add_cofactors_list

[]


In [14]:
filename = 'protein_modification.txt'
file = open(filename,'a')

cplx_cofactor_dict = dict()
RXNS = bsub.reactions
for rxn in RXNS.instances:
    
    rxn_data = pc.PToolsFrame.PFrame.get_frame_data(rxn)
    
    substrates = rxn_data.substrates
    rxn_cofactors = list(set(cofactors) & set(substrates))
    
    # print rxn_data.frameid, substrates
    if any(rxn_cofactors):
        cplx_list = pc.PGDB.enzymes_of_reaction(bsub,rxn)
        for cplx in cplx_list:
            cplx = str(cplx[1:len(cplx)-1])
            
            if (cplx not in cplx_cofactor_dict.keys()):
                cplx_cofactor_dict[cplx] = []
                
            for cofactor in rxn_cofactors:
                
                if (cofactors[cofactor] not in cplx_cofactor_dict[cplx]):
                    cplx_cofactor_dict[cplx].append(cofactors[cofactor])
                    
for cplx in cplx_cofactor_dict.keys():
    cofactor_string = ''
    mod_string = ''
    for cofactor in cplx_cofactor_dict[cplx]:
        cofactor_string = cofactor_string + cofactor + '()' + ' AND '
        mod_string = mod_string + '_mod_' + cofactor
    cofactor_string = cofactor_string[0:len(cofactor_string)-5]
    
    string = cplx + mod_string +'\t' + cplx + '\t' + cofactor_string + '\t' + 'M_protein_recon' + '\n'
    
    file.write(string)
file.close()


## Cleaved methionine

In [15]:
## Methionine cleaved feature dictionary
met_cleaved_features = []
for instance in bsub.amino_acid_sites.instances:
    instance_data = pc.PToolsFrame.PFrame.get_frame_data(instance)
    try :
        if instance.comment[0] == 'UniProt: Removed.':
            met_cleaved_features.append(instance.frameid)
    except:
        continue

KeyboardInterrupt: 

In [ ]:
met_cleaved_features

In [ ]:
met_cleaved_prots = []
for protein in bsub.proteins.instances:
    protein_id = frameid_to_str(protein.frameid)
    gene_id = protein_id.split('-MONOMER')[0]
    try:
        features = protein.features
        if list(set(features) & set(met_cleaved_features)):
            met_cleaved_prots.append(gene_id)
    except:
        continue

In [ ]:
string = ''
i = 0
for prot in met_cleaved_prots:
    i = i + 1
    string = string + "'" + prot + "'" + ','
    if not i%5:
        string = string + '\n'
print string

## peptide_compartment_and_pathways

In [ ]:
membrane_monomers = []
for protein in bsub.proteins.instances:
    protein_id = frameid_to_str(protein.frameid)
    protein_data = pc.PToolsFrame.PFrame.get_frame_data(protein)
    
    locations = protein_data.locations
    if locations is not None:
        location = frameid_to_str(locations[0])
        if location == 'CCI-PM-BAC-POS-GP':
            protein_id = protein_id.split('-MONOMER')[0]
            membrane_monomers.append(protein_id)
membrane_monomers

In [ ]:
filename = 'peptide_compartment_and_pathways.txt'
file = open(filename,'w')

file.write('Complex' + '\t' + 'Complex_compartment' + '\t' + 'Protein' + '\t' + 'Protein_compartment' +
           '\t' + 'translocase_pathway''\n')

for protein in bsub.proteins.instances:
    protein_id = frameid_to_str(protein.frameid)
    protein_data = pc.PToolsFrame.PFrame.get_frame_data(protein)
    
    genes_of_cplx = pc.PGDB.genes_of_protein(bsub,protein)
    if (genes_of_cplx):
        test_gene = frameid_to_str(genes_of_cplx[0])
        if (test_gene in membrane_monomers):
            print test_gene
            for gene in protein_complexes_dict[protein_id]:
                file.write(protein_id + '\t' + 'Membrane' + '\t' + gene + '(1)' + '\t' +
                               'Membrane' + '\t' + 'y' + '\n')
file.close()

In [ ]:
for protein in bsub.proteins.instances:
    protein_id = frameid_to_str(protein.frameid)
    protein_data = pc.PToolsFrame.PFrame.get_frame_data(protein)
    if protein_id == 'CPLX8J2-52':
        print protein_data
        break